Models
====
We want to understand:
 - how much the completion and compliance rates change per day
 - if there is a difference between male/female participants

To do this we'll use mixed-effects models.

This notebook will:
 - Create a .csv file holding the data
 - Run an R script that will perform the fits and create some plots
 - Display these plots and the associated odds ratios and confidence intervals

In [ ]:
"""
Create a dataframe showing how many prompts and responses each user had per day, grouping nearby prompts so
that we have the expected (roughly) 12 prompts per day.

"""

import numpy as np
import pandas as pd

from analysis_utils import clean


def collapse_meal_info(meal_df: pd.DataFrame, delta: pd.Timedelta) -> pd.DataFrame:
    """
    Find the meal entries that are close enough to each other to be responses to the same prompt.

    Must have a "responded?" column that is 1 if the prompt was responded to, and 0 otherwise.
    Successive entries that are less than delta apart and have the same "responded?" value are considered to
    be a response to the same prompt.

    :param meal_df: a dataframe with a DateTimeIndex that includes an "reponded?" column
    :param delta: the maximum time difference between two entries for them to be considered the same prompt

    """
    collapsed_meal_info = pd.DataFrame()

    for _, group in meal_df.groupby("p_id"):
        assert group.index.is_monotonic_increasing

        # Mark which ones are near enough each other to be considered the same
        n_entries = len(group)
        keep = np.ones(n_entries, dtype=bool)

        for i in range(1, n_entries):
            if (group.index[i] - group.index[i - 1] < delta) and (
                group["responded?"].iloc[i] == group["responded?"].iloc[i - 1]
            ):
                keep[i] = False

        # Append to the new dataframe
        collapsed_meal_info = pd.concat([collapsed_meal_info, group[keep]])

    return collapsed_meal_info


# Read the smartwatch entries
meal_info = clean.cleaned_smartwatch(keep_catchups=False, keep_day0=False)[
    ["p_id", "delta", "meal_type"]
]

# Turn delta into a number of days
meal_info["day"] = meal_info["delta"].dt.days
meal_info.drop(columns=["delta"], inplace=True)

# Turn meal type into a binary variable for whether it was a response or non-response
meal_info["responded?"] = (
    meal_info["meal_type"].isin({"Meal", "Drink", "Snack", "No food/drink"}).astype(int)
)
meal_info.drop(columns=["meal_type"], inplace=True)

# Collapse nearby entries of the same type
# Using a 27 minute window because that gives us roughly the expected number of prompts per day
# Using a larger time window doesn't change anything; using a smaller time window leaves us with some
# days with much more than 12 prompts
meal_info = collapse_meal_info(meal_info, pd.Timedelta(minutes=27))

# Read the survey data
demographic_df = clean.cleaned_survey()[
    [
        "residents_id",
        "respondent_sex",
        "respondent_ethnicity",
        "age_dob",
        "phyactq1",  # In the last 7 days, how many days did you attend school?
        "smart1_7to9",  # Did your child participate in the smartwatch study?
        "smart1_10to17",  # Did you participate in the smartwatch study?
    ]
]
demographic_df.rename(
    columns={
        "residents_id": "p_id",
        "age_dob": "age",
        "phyactq1": "schooldays",
        "respondent_sex": "sex",
        "respondent_ethnicity": "ethnicity",
    },
    inplace=True,
)

# Convert sex into 0 or 1 in
demographic_df["sex"] -= 1

# Merge them into one
joined_df = meal_info.reset_index().merge(
    demographic_df, left_on="p_id", right_on="p_id", how="left"
)

# Only keep participants who wore the smartwatch
keep = (joined_df["smart1_7to9"] == 1) | (joined_df["smart1_10to17"] == 1)
print(f"Keeping {len(joined_df.loc[keep, 'p_id'].unique())} participants")
joined_df = joined_df.loc[keep]
joined_df.drop(columns=["smart1_7to9", "smart1_10to17"], inplace=True)

# Group by p_id and day, count how many responses and prompts there were
completion_df = pd.DataFrame(
    columns=[
        "p_id",
        "day",
        "n_prompts",
        "n_responses",
        "sex",
        "ethnicity",
        "age",
        "schooldays",
    ]
)
for i, (_, group) in enumerate(joined_df.groupby(["p_id", "day"])):
    n_prompts = len(group["responded?"])
    n_responses = group["responded?"].sum()

    # Clip these to 12
    if n_prompts > 12:
        print(
            f"Participant {group['p_id'].iloc[0]} had {n_prompts} prompts on day {group['day'].iloc[0]}"
        )
        n_prompts = 12

    if n_responses > 12:
        print(
            f"Participant {group['p_id'].iloc[0]} had {n_responses} responses on day {group['day'].iloc[0]}"
        )
        n_responses = 12

    completion_df = pd.concat(
        [
            completion_df,
            pd.DataFrame(
                {
                    "p_id": group["p_id"].iloc[0],
                    "day": group["day"].iloc[0],
                    "n_prompts": n_prompts,
                    "n_responses": n_responses,
                    "sex": group["sex"].iloc[0],
                    "ethnicity": group["ethnicity"].iloc[0],
                    "age": group["age"].iloc[0],
                    "schooldays": group["schooldays"].iloc[0],
                },
                index=[i],
            ),
        ]
    )

# Turn the age into age groups
completion_df["age_group"] = (completion_df["age"] > 12).astype(int)

completion_df.to_csv("outputs/data/compliance.csv", index=False)

Now that we have saved this csv file, we'll read it in using R and run the linear models:

In [ ]:
"""
Run an R script to fit linear models to the compliance and completion rates,
and save the results in a text file

"""
import subprocess

subprocess.run(["Rscript", "analysis_utils/r/binomial_models.R"], check=True)

We've now run the models and created some plots, so now let's read the model output (from the text files that the R script generated) and find the associated odds ratios and 95% confidence intervals

In [ ]:
"""
Interpret the results of the compliance model - find the odds ratio for the drop-off per day

"""

import pathlib

from IPython.display import display, Image


def lines2df(lines: list[str]) -> pd.DataFrame:
    """
    Convert a list of strings into a dataframe

    """
    retval = pd.DataFrame()
    # Cut off the table header and --- indicating the end of the table
    for line in lines[1:-1]:
        param, estimate, err, *_, sig = line.split()

        # No significance
        if set(sig) != {"*"}:
            sig = ""

        # Add a row where the index is param, and the columns are estimate and err
        retval = pd.concat(
            [
                retval,
                pd.DataFrame(
                    {"estimate": [float(estimate)], "err": [float(err)], "sig": [sig]},
                    index=[param],
                ),
            ]
        )

    return retval


def read_fixed_effects(path: str) -> pd.DataFrame:
    """
    Read the table of fixed effects from an R model summary into a dataframe

    """
    keep = False
    table_lines = []
    with open(path) as f:
        for line in f.readlines():
            # Keep only the lines between "Fixed effects:" and "---"
            if keep:
                table_lines.append(line)
            if line == "Fixed effects:\n":
                keep = True
            elif line == "---\n":
                keep = False

    df = lines2df(table_lines)
    df.columns.name = pathlib.Path(path).stem
    return df


def show_odds_ratio(df: pd.DataFrame) -> None:
    # Dataframe with the odds ratio
    retval = np.exp(df[["estimate"]])

    # Find the confidence interval
    retval["lower"] = np.exp(df["estimate"] - 1.96 * df["err"])
    retval["upper"] = np.exp(df["estimate"] + 1.96 * df["err"])

    retval.columns.name = "Odds Ratio"

    retval["sig"] = df["sig"]

    display(retval)


compliance_effects = read_fixed_effects("outputs/imgs/binomial/compliance_model.txt")
display(compliance_effects)

# exp(Intercept) gives the odds of compliance on day 0
# exp(day) gives the odds ratio for compliance for each additional day
show_odds_ratio(compliance_effects)
Image("outputs/imgs/binomial/compliance_model.png")


In [ ]:
"""
Do the same with the completion model

"""
completion_effects = read_fixed_effects("outputs/imgs/binomial/completion_model.txt")
display(completion_effects)
show_odds_ratio(completion_effects)
Image("outputs/imgs/binomial/completion_model.png")

In [ ]:
"""
Interpret the difference between sexes in the compliance model

"""
# exp(sex) gives the odds ratio for females compared to males
# exp(day:sex) gives the odds ratio for the interaction
compliance_sex_effects = read_fixed_effects("outputs/imgs/binomial/compliance_sex_model.txt")
display(compliance_sex_effects)
show_odds_ratio(compliance_sex_effects)
Image("outputs/imgs/binomial/compliance_sex_model.png", width=500)

In [ ]:
"""
Do the same with the completion model

"""
completion_sex_effects = read_fixed_effects("outputs/imgs/binomial/completion_sex_model.txt")
display(completion_sex_effects)

np.exp(completion_sex_effects[["estimate", "err"]])
show_odds_ratio(completion_sex_effects)
Image("outputs/imgs/binomial/completion_sex_model.png")

In [ ]:
"""
Do the same with the age models

"""
compliance_age_effects = read_fixed_effects("outputs/imgs/binomial/compliance_age_model.txt")
display(compliance_age_effects)

np.exp(compliance_age_effects[["estimate", "err"]])
show_odds_ratio(compliance_age_effects)
Image("outputs/imgs/binomial/compliance_age_model.png")

In [ ]:
completion_age_effects = read_fixed_effects("outputs/imgs/binomial/completion_age_model.txt")
display(completion_age_effects)

np.exp(completion_age_effects[["estimate", "err"]])
show_odds_ratio(completion_age_effects)
Image("outputs/imgs/binomial/completion_age_model.png")